# Transfer Learning Approach

In [1]:
## Train a model to classify the images
#
## Import libraries

import os
import numpy as np

from tensorflow import keras
np.random.seed(0)

import src.config as config
from src.dataloader import read_train, read_val
from src.preprocessing import CustomDataGenerator
from src.results import plot_history, plot_cm, class_report,\
                            save_params, write_summary
from src.utils import setup_logger, header
import argparse
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.regularizers import l2,l1,l1_l2

RAW_DATA_PATH = 'input/data/raw'
META_PATH = 'input/metadata/'

## Set up constants
LEARNING_RATE = .0001
BATCH_SIZE = 64
EPOCHS = 10
EXPNAME = "experiment"
EVENTNAME = f"lr_{LEARNING_RATE}_bs_{BATCH_SIZE}_ep_{EPOCHS}"
OUTPUT_FOLDER = "notebooks/transfer/output"


#Readind data directly from the csv file and raw images in order to use data augmentation
df_labels = pd.read_csv(os.path.join(META_PATH, 'MAMe_labels.csv'), header=None, names=['id', 'label'])
df_labels['label'] = df_labels['label'].str.strip()
df_info = pd.read_csv(META_PATH + 'MAMe_dataset.csv')
df_info["Medium"] = df_info["Medium"].str.strip()
df_load_data = df_info.merge(df_labels, right_on='label', left_on='Medium')[['Image file', 'Subset', 'Medium']]

df_load_data

2023-11-24 17:39:17.351877: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 17:39:17.376134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 17:39:17.737649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


,Image file,Subset,Medium
0,436018.jpg,train,Oil on canvas
1,11779.jpg,train,Oil on canvas
2,19022.jpg,train,Oil on canvas
3,435883.jpg,train,Oil on canvas
4,10481.jpg,train,Oil on canvas
...,...,...,...
37402,261318.jpg,test,Albumen photograph
37403,646703.jpg,test,Albumen photograph
37404,417445.jpg,test,Albumen photograph
37405,646328.jpg,test,Albumen photograph


In [2]:
from keras import applications
model = applications.VGG16(weights = "imagenet", include_top=True)

# Change output layer
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

output_shape = len(config.LABELS)

x = model.layers[-2].output
predictions = Dense(output_shape, activation="softmax")(x)
model = Model(inputs=model.input, outputs=predictions)
model.summary()


2023-11-24 17:39:18.600439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-24 17:39:18.616226: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-24 17:39:18.616342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [30]:
from src.dataloader import read_train, read_val
from src.preprocessing import CustomDataGenerator
import cv2

# X_train, y_train = read_train()
X_val, y_val = read_val()

X_val_resized = np.array([cv2.resize(img / 255. , (224, 224)) for img in X_val])

In [31]:
from keras.applications.vgg16 import preprocess_input
X_val_resized = preprocess_input(X_val_resized)

In [33]:
y_pred_vgg16 = model.predict(X_val_resized)

46/46 [==============================] - 4s 84ms/step


In [34]:
y_pred_vgg16_lab = np.argmax(y_pred_vgg16, axis=1)
accuracy_vgg16 = np.mean(y_pred_vgg16_lab == y_val)
print(f"Accuracy: {accuracy_vgg16:.4f}")

Accuracy: 0.0345
